# **PDF QA Assistant**
Using LangChain, ChromaDB, and Groq's LLM

In [1]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.49
    Uninstalling langchain-core-0.3.49:
      Successfully uninstalled langchain-core-0.3.49
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [2]:
!pip install langchain_groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 3.8 MB/s eta 0:00:00


In [3]:
import os
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings  # Changed from HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQA

In [5]:
# Groq API Key Configuration
from getpass import getpass
# Prompt the user to input the API key securely
GROQ_API_KEY = getpass("Enter your Groq API key:")

Enter your Groq API key:··········


In [8]:
import os
print(os.path.exists('/content/Introduction_GenAI_EoAI.pdf'))


True


In [9]:
def load_pdf(file_path):
    """
    Load PDF using PyPDFLoader for better PDF parsing
    """
    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
        print(f"Successfully loaded PDF: {file_path}")
        print(f"Total pages loaded: {len(documents)}")
        return documents
    except Exception as e:
        print(f"Error loading PDF: {e}")
        return []

In [10]:
# Vector Database Configuration
VECTOR_DB_NAME = "pdf_embeddings"  # Specific name for the vector database

In [11]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 4.8 MB/s eta 0:00:00


In [13]:
# Load documents
documents = load_pdf('/content/Introduction_GenAI_EoAI.pdf')  # Replace with your document path

Successfully loaded PDF: /content/Introduction_GenAI_EoAI.pdf
Total pages loaded: 69


In [14]:
# Check if documents were loaded
if not documents:
    print("No documents to process. Exiting.")
    exit()

In [15]:
# Text splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
texts = text_splitter.split_documents(documents)

print(f"Total text chunks created: {len(texts)}")

Total text chunks created: 71


In [16]:
# Correctly instantiate Hugging Face Embeddings
try:
    # Use a more widely supported embedding model
    embeddings = HuggingFaceEmbeddings(
        model_name="all-MiniLM-L6-v2",  # Lightweight, widely compatible model
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
except Exception as e:
    print(f"Error loading embeddings: {e}")
    exit()

<ipython-input-16-8af36e6d0cab>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:00


In [18]:
# Create vector store with a specific persistent directory
vector_db_path = os.path.join(".", VECTOR_DB_NAME)

try:
    # Ensure the directory exists
    os.makedirs(vector_db_path, exist_ok=True)

    # Create vector database
    vectorstore = Chroma.from_documents(
        texts,
        embeddings,
        persist_directory=vector_db_path
    )

    # Verify vector database creation
    print(f"Vector Database '{VECTOR_DB_NAME}' created successfully!")
    print(f"Persistent directory: {vector_db_path}")

    # Create a retrieval QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=ChatGroq(
            model="llama3-8b-8192",
            temperature=0.2,
            groq_api_key=GROQ_API_KEY
        ),
        retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
        chain_type="stuff"  # Most straightforward chain type
    )

    # Function to ask questions about the PDF
    def ask_pdf_question(question):
        try:
            # Retrieve and answer the question
            response = qa_chain.invoke(question)
            return response['result']
        except Exception as e:
            return f"An error occurred: {e}"

    # Example usage
    print("\n--- PDF Question Answering Demo ---")

    # Interactive question-asking loop
    while True:
        user_question = input("\nAsk a question about the PDF (or type 'exit' to quit): ")

        if user_question.lower() == 'exit':
            break

        if user_question:
            answer = ask_pdf_question(user_question)
            print("\nAnswer:", answer)

except Exception as e:
    print(f"Error creating vector database: {e}")
    exit()

Vector Database 'pdf_embeddings' created successfully!
Persistent directory: ./pdf_embeddings

--- PDF Question Answering Demo ---

Ask a question about the PDF (or type 'exit' to quit): Why Gen AI now?

Answer: According to the provided context, the reasons why GenAI is now possible include:

1. Availability of large and diverse datasets
2. AI models learn patterns, correlations, and characteristics of large datasets
3. Pre-trained state-of-the-art models
4. Advancements in hardware; GPUs
5. Access to cloud computing
6. Open-source software, Hugging Face
7. Generative Adversarial Networks (GANs)
8. Transformers Architecture
9. Reinforcement Learning from human feedback (RLHF)

Ask a question about the PDF (or type 'exit' to quit): why NLP is hard?

Answer: According to the context, NLP is hard due to several reasons:

1. Ambiguity
2. Scale
3. Variation
4. Expressivity
5. Unknown representation

Additionally, the context also mentions that language consists of many levels of linguistic